## G Colab

In [1]:
colab_env = False

In [ ]:
print('Mounting Gdrive')
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/MyDrive/')


colab_env = True

## Lib

In [2]:
import os
import tensorflow as tf
import keras
import pickle

from mlflow import MlflowClient
import mlflow.tensorflow

2024-04-25 10:55:39.200587: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-25 10:55:40.108853: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-25 10:55:41.106739: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
tf.config.experimental.list_physical_devices('GPU')

2024-04-25 09:25:54.275700: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-25 09:25:54.312990: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-25 09:25:54.313137: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Vars

In [3]:
epochs = 25
learning_rate = 1e-4

In [4]:
if colab_env == True:
    model_path = "/content/drive/MyDrive/ml_projects/mushroom-project/storage/models/builded/generated_model_efficientnetv2_21k_finetuned_1k_v1.keras"
    train_dataset_path = "/content/drive/MyDrive/ml_projects/mushroom-project/storage/datas/tf_datasets/train_dataset"
    val_dataset_path = "/content/drive/MyDrive/ml_projects/mushroom-project/storage/datas/tf_datasets/val_dataset"
    tracking_uri = "/content/drive/MyDrive/ml_projects/mushroom-project/storage/mlruns"
    model_checkpoint_url = "./content/drive/MyDrive/ml_projects/mushroom-project/storage/models/model_checkpoint"    

else:
    model_path = "../../storage/models/builded/generated_model_efficientnetv2_21k_finetuned_1k_v1.keras"
    train_dataset_path = "../../storage/datas/tf_datasets/train_dataset"
    val_dataset_path = "../../storage/datas/tf_datasets/val_dataset"
    tracking_uri = "../../storage/mlruns"
    model_checkpoint_url = "../../storage/models/model_checkpoint"

## Load model & Datasets

### Model

In [5]:
# Load Model
model = keras.saving.load_model(model_path)

2024-04-25 10:55:52.408320: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-25 10:55:52.556290: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [6]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetv2-b0 (Functional)  │ (None, 7, 7, 1280)     │     5,919,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │     1,311,744 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,889,754 (30.10 MB)

 Trainable params: 1,970,442 (7.52 MB)

 Non-trainable params: 5,919,312 (22.58 MB)

In [7]:
# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

### Datasets

In [8]:
# Load datasets
train_datas = tf.data.Dataset.load(train_dataset_path)
val_datas = tf.data.Dataset.load(val_dataset_path)

## Fit

In [10]:
# Callbacks
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    os.path.join(model_checkpoint_url, "model_{epoch:02d}_{val_loss:.2f}.weights.h5"),
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
    mode='auto',
    save_freq='epoch',
    initial_value_threshold=None)

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=3,
    verbose=1,
    mode='auto',
    baseline=None,
    restore_best_weights=True,
    start_from_epoch=5)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=3,
    verbose=1,
    mode='auto',
    min_delta=0.0001,
    cooldown=0)

In [11]:
# Start mlflow client
mlflow.set_tracking_uri(tracking_uri)
client = MlflowClient()
mlflow.tensorflow.autolog()

history = model.fit(
    train_datas,
    validation_data = val_datas,
    epochs = epochs,
    initial_epoch = 0,
    callbacks=[model_checkpoint, early_stopping, reduce_lr],
    verbose = 1)

Epoch 1/25
3145/3145 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step - accuracy: 0.6219 - loss: 1.1073 - top_k_categorical_accuracy: 0.8643
Epoch 1: val_loss improved from inf to 0.72751, saving model to ../../storage/models/model_checkpoint/model_01_0.73.weights.h5
3145/3145 ━━━━━━━━━━━━━━━━━━━━ 402s 126ms/step - accuracy: 0.6219 - loss: 1.1073 - top_k_categorical_accuracy: 0.8643 - val_accuracy: 0.7671 - val_loss: 0.7275 - val_top_k_categorical_accuracy: 0.9380 - learning_rate: 1.0000e-04
Epoch 2/25
3145/3145 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step - accuracy: 0.7743 - loss: 0.6604 - top_k_categorical_accuracy: 0.9494
Epoch 2: val_loss did not improve from 0.72751
3145/3145 ━━━━━━━━━━━━━━━━━━━━ 424s 135ms/step - accuracy: 0.7743 - loss: 0.6604 - top_k_categorical_accuracy: 0.9494 - val_accuracy: 0.7544 - val_loss: 0.7348 - val_top_k_categorical_accuracy: 0.9364 - learning_rate: 1.0000e-04
Epoch 3/25
3145/3145 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step - accuracy: 0.8027 - loss: 0.5755 - top_k_categorical_accu

2024/04/25 11:54:59 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during tensorflow autologging: Could not find experiment with ID 0


In [12]:
# Save Model
trained_model_path = "../../storage/models/trained/generated_model_efficientnetv2_21k_finetuned_1k_v1_trained.keras"
model.save(filepath = trained_model_path, overwrite = True)

# Save history
history_path = "../../storage/models/histories/generated_model_efficientnetv2_21k_finetuned_1k_v1_trained_history.pickle"
with open(history_path, 'wb') as file:
    pickle.dump(history.history, file)

### Fine tuning

In [13]:
fine_tuned_model = model

for layer in fine_tuned_model.layers:
    layer.trainable = False
    
fine_tuned_model.layers[1].trainable = True

fine_tuned_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 1e-6),
              loss='categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=3)])

In [14]:
history_fine_tune = model.fit(
    train_datas,
    validation_data = val_datas,
    epochs = epochs-15,
    initial_epoch = 0,
    callbacks=[model_checkpoint, early_stopping, reduce_lr],
    verbose = 1)

Epoch 1/10
3145/3145 ━━━━━━━━━━━━━━━━━━━━ 0s 568ms/step - accuracy: 0.5254 - loss: 1.6994 - top_k_categorical_accuracy: 0.8022
Epoch 1: val_loss did not improve from 0.71969
3145/3145 ━━━━━━━━━━━━━━━━━━━━ 1849s 581ms/step - accuracy: 0.5254 - loss: 1.6993 - top_k_categorical_accuracy: 0.8022 - val_accuracy: 0.6296 - val_loss: 1.2190 - val_top_k_categorical_accuracy: 0.8688 - learning_rate: 1.0000e-06
Epoch 2/10
3145/3145 ━━━━━━━━━━━━━━━━━━━━ 0s 584ms/step - accuracy: 0.6232 - loss: 1.2565 - top_k_categorical_accuracy: 0.8696
Epoch 2: val_loss did not improve from 0.71969
3145/3145 ━━━━━━━━━━━━━━━━━━━━ 1877s 597ms/step - accuracy: 0.6232 - loss: 1.2564 - top_k_categorical_accuracy: 0.8696 - val_accuracy: 0.6709 - val_loss: 1.0442 - val_top_k_categorical_accuracy: 0.8983 - learning_rate: 1.0000e-06
Epoch 3/10
3145/3145 ━━━━━━━━━━━━━━━━━━━━ 0s 589ms/step - accuracy: 0.6775 - loss: 1.0373 - top_k_categorical_accuracy: 0.9017
Epoch 3: val_loss did not improve from 0.71969
3145/3145 ━━━━━━━━

2024/04/25 16:41:32 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during tensorflow autologging: Could not find experiment with ID 0


In [15]:
# Save model
fine_tuned_model_path = "../../storage/models/trained/generated_model_efficientnetv2_21k_finetuned_1k_v1_trained_fine_tuned.keras"
fine_tuned_model.save(filepath = fine_tuned_model_path, overwrite = True)

history_fine_tune_path = "../../storage/models/histories/generated_model_efficientnetv2_21k_finetuned_1k_v1_trained_fine_tuned_history.pickle"
with open(history_fine_tune_path, 'wb') as file:
    pickle.dump(history_fine_tune.history, file)